# This notebook is an introduction to the data derived storms addition into geoclaw

### Given storm data in the form of wind and pressure files that are in a fixed width fortran format
### ie Oceanweather inc format (taken from https://wiki.adcirc.org/NWS12)
OWI WWS Wind Output Ucomp,Vcomp in m/s           Start:1995060600 End:1995060600

iLat=  67iLong=  67DX= 1.250DY=  .833SWLat=  22.500SWlon= -82.500Dt=199506060000
 -1.16856  -1.06439   -.84875  -1.03460  -1.50047  -2.09462  -2.80243  -3.55863
 -4.24125  -4.84273  -5.59486  -5.37088  -5.30224  -5.12534  -4.89537  -4.67412
 -4.49203  -4.35772  -4.26612  -4.20260  -4.14746  -4.08396  -4.00686  -3.92213
 -3.83615  -3.74765  -3.65182  -3.54998  -3.45299  -3.37660  -3.32959  -3.28037
 -3.10631  -2.67723  -2.08363  -1.53773  -1.12623   -.83526   -.62870   -.47371
 [rest deleted]

In [6]:
# We can convert this to a netcdf format to be run in geoclaw.
from clawpack.geoclaw.surge import storm, data_storms
import os
import numpy as np

In [8]:
# Initialize the data storm class, providing the file names for the wind and pressure files
storm_data = storm.DataDerivedStorms(os.path.join('./', 'isaac'), wind_file_ext='WIN', pressure_file_ext='PRE')

<class 'list'> 117012
<class 'list'> 58506


In [9]:
# Convert data to output needed for netcdf file
storm_data.parse_data()

In [12]:
# Write to netcdf, only need to supply the name of the output file you want
# THe code adds the .nc at the end
storm_data.write_data_derived('isaac')

In [13]:
# Lets look at the data format
import xarray as xr
ds = xr.open_dataset('isaac.nc')
ds

<xarray.Dataset>
Dimensions:   (time: 42, lat: 96, lon: 116)
Coordinates:
  * lat       (lat) float64 8.0 8.25 8.5 8.75 9.0 ... 31.0 31.25 31.5 31.75
  * lon       (lon) float64 -99.0 -98.75 -98.5 -98.25 ... -70.75 -70.5 -70.25
  * time      (time) float64 0.0 2.16e+04 4.32e+04 ... 8.64e+05 8.856e+05
Data variables:
    u         (time, lat, lon) float64 ...
    v         (time, lat, lon) float64 ...
    speed     (time, lat, lon) float64 ...
    pressure  (time, lat, lon) float64 ...

You can see that there is 42 time steps saved, each time step has a 2d array of shape(lat,lon) for the horizontal and vertical wind velocities the wind speed and pressure

In the data_storms_module.f90 this data is read into a storm object and as the simulation progresses it first interpolates each data point between two points in time, then uses bilinear interpolation to interpolate teh data spatially as well.